In [1]:
#!pip install bertopic

# Datathon 2024

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/My Drive/Datathon/df_insigths.csv'
df = pd.read_csv(file_path)
df['date'] = pd.to_datetime(df['date'])
print(df.shape)
df.info()

(812, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 812 entries, 0 to 811
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         812 non-null    datetime64[ns]
 1   tweet        812 non-null    object        
 2   sentiment    812 non-null    object        
 3   feedback     686 non-null    object        
 4   competencia  16 non-null     object        
 5   problemas    49 non-null     object        
 6   leads        101 non-null    object        
dtypes: datetime64[ns](1), object(6)
memory usage: 44.5+ KB


In [ ]:
df

,date,tweet,sentiment,feedback,competencia,problemas,leads
0,2023-01-01,"Resuelto, muchísimas gracias , excelente servi...",positivo,"Resuelto, muchísimas gracias , excelente servi...",NaN,NaN,NaN
1,2023-01-02,"Muchas gracias, espero su dm",Neutro,El cliente menciona que le gustaría recibir má...,NaN,NaN,cuenta de ahorros para jóvenes
2,2023-01-02,Muchas gracias!,positivo,El servicio al cliente de Hey Banco es excelen...,NaN,NaN,NaN
3,2023-01-02,Algo similar me paso. Quería renovar mi token ...,negativo,Dificultad para renovar el token y realizar un...,NaN,Dificultad para renovar el token y realizar un...,"transferencia, cerrar la cuenta"
4,2023-01-02,Yeeeei! a través de mi cuenta en acabo de cont...,positivo,La herramienta de domiciliación de cuentas es ...,NaN,NaN,"cuenta de ahorro, domiciliar, ahorrar discipli..."
...,...,...,...,...,...,...,...
807,2024-04-21,"Ahora confirmo, gracias estimado",Neutro,El cliente agradece el servicio recibido en He...,NaN,NaN,NaN
808,2024-04-23,😔 de acuerdo!! Muchas gracias.,negativo,NaN,NaN,NaN,NaN
809,2024-04-23,"Tío cuando me van a graduar, tengo la TDC gara...",negativo,El cliente expresa frustración por no haber si...,NaN,El problema principal señalado es la falta de ...,NaN
810,2024-04-25,"una duda, con quién tengo que ver si tengo pro...",negativo,El cliente tiene problemas para aumentar la lí...,NaN,Señala claramente el problema con la aplicació...,NaN


# Visualización de Tópicos por Sentimiento

In [ ]:
stop_words = set(stopwords.words('spanish'))
def remove_stopwords(text):
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if not word.lower() in stop_words]
    return ' '.join(filtered_text)

In [ ]:
docs = df[df['sentiment']=='positivo']['leads'].dropna().values
cdocs = [remove_stopwords(text) for text in docs]

In [ ]:
embed_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.0, metric='cosine') #n_neig=3, ncomp=3
hd_model = HDBSCAN(min_cluster_size = 3, metric='chebyshev')#, prediction_data=True) #min_clust=2

In [ ]:
topic_model = BERTopic(embedding_model= embed_model,
                       umap_model = umap_model,
                       hdbscan_model = hd_model,
                       top_n_words=3,
                       verbose=True, nr_topics="auto")
topics, probs = topic_model.fit_transform(cdocs)
new_topics = topic_model.reduce_outliers(cdocs, topics)
topic_model.update_topics(cdocs, topics=new_topics)

2024-05-05 07:03:23,084 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2024-05-05 07:03:24,956 - BERTopic - Embedding - Completed ✓
2024-05-05 07:03:24,958 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-05 07:03:28,838 - BERTopic - Dimensionality - Completed ✓
2024-05-05 07:03:28,843 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-05-05 07:03:28,853 - BERTopic - Cluster - Completed ✓
2024-05-05 07:03:28,856 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-05-05 07:03:28,898 - BERTopic - Representation - Completed ✓
2024-05-05 07:03:28,905 - BERTopic - Topic reduction - Reducing number of topics
2024-05-05 07:03:28,953 - BERTopic - Topic reduction - Reduced number of topics from 13 to 9
100%|██████████| 1/1 [00:00<00:00, 337.87it/s]
2024-05-05 07:03:29,024 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pi

In [ ]:
## Run the visualization with the original embeddings
topic_model.visualize_documents(cdocs)#, embeddings=embeddings)

In [ ]:
cdocs

['Resuelto , muchísimas gracias , excelente servicio siempre',
 'servicio cliente Hey Banco excelente , siempre resuelven dudas manera rápida eficiente .',
 'herramienta domiciliación cuentas útil ahorrar manera disciplinada .',
 'servicio cliente Hey Banco excelente , siempre resuelven dudas manera rápida eficiente .',
 'servicio cliente Hey Banco excelente , siempre resuelven dudas manera rápida eficiente .',
 'Recomendación inversión Hey Banco comentario positivo rendimientos . Mención límite inversión banco .',
 'Agradecimiento playera recibida Galerías GDL',
 'servicio cliente Hey Banco excelente , siempre resuelven dudas manera rápida eficiente .',
 'Respetos ustedes ... Gracias servicio 🙌🏻',
 'cliente pudo realizar depósito OXXO después haber sido bloqueado desbloqueado , agradece ayuda brindada .',
 'cliente expresa sorpresa alegría haber ganado .',
 'cliente considera Hey Banco buena opción invertir obtener buen rendimiento .',
 'cliente menciona pudo pagar servicio internet C

In [ ]:
docs = df[df['sentiment']=='positivo']['feedback'].dropna().values
cdocs = [remove_stopwords(text) for text in docs]

In [ ]:
embed_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
umap_model = UMAP(n_neighbors=7, n_components=8, min_dist=0.0, metric='cosine')
hd_model = HDBSCAN(min_cluster_size = 7, metric='euclidean')#, prediction_data=True) #min_clust=2

In [ ]:
topic_model = BERTopic(embedding_model= embed_model,
                       umap_model = umap_model,
                       hdbscan_model = hd_model,
                       top_n_words=2,
                       verbose=True, nr_topics="auto")
topics, probs = topic_model.fit_transform(cdocs)
#new_topics = topic_model.reduce_outliers(cdocs, topics)
#topic_model.update_topics(cdocs, topics=new_topics)

2024-05-05 08:12:45,583 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/18 [00:00<?, ?it/s]

2024-05-05 08:12:58,226 - BERTopic - Embedding - Completed ✓
2024-05-05 08:12:58,230 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-05 08:13:01,662 - BERTopic - Dimensionality - Completed ✓
2024-05-05 08:13:01,665 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-05-05 08:13:01,685 - BERTopic - Cluster - Completed ✓
2024-05-05 08:13:01,687 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-05-05 08:13:01,728 - BERTopic - Representation - Completed ✓
2024-05-05 08:13:01,730 - BERTopic - Topic reduction - Reducing number of topics
2024-05-05 08:13:01,789 - BERTopic - Topic reduction - Reduced number of topics from 19 to 14


In [ ]:
## Run the visualization with the original embeddings
topic_model.visualize_documents(cdocs)#, embeddings=embeddings)